## Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras import Sequential
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import backend
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.callbacks import History
import seaborn as sns
import time as time
sns.set( color_codes=True)
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.initializers import glorot_uniform
from copy import copy
import statsmodels.api as sm
import scipy.stats as si
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
months = np.arange(1, 13, step = 1)
print(months)
years = np.arange(2006, 2022, step = 1 )
print(years)

In [ ]:
from datetime import datetime, date
from dateutil.relativedelta import relativedelta, MO

In [ ]:
import random

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
def bsmcall(S, K, T, r,q, sigma):
    d1 = (np.log(S/K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T)) 
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q*T) * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)

In [ ]:
from scipy.optimize import minimize, leastsq

from keras.utils.vis_utils import plot_model

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
import math

In [ ]:
from keras.callbacks import EarlyStopping
earlyStop=EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=2)

##MLP

In [ ]:
# defining the different models

def mlp(x_train,x_test,y_train,y_test,y,vol_train,vol_test,scaler_y,BSM_train,BSM_test,optimalvalues):
  from pathlib import Path  

  for k in range(1):
    mse_test,  mse_BSM_test = np.zeros(5), np.zeros(5)
    rmse_test  , rmse_BSM_test = np. zeros (5) , np. zeros (5) 
    R2_test  , R2_BSM_test = np. zeros (5) , np. zeros (5) 
    diff_test ,  diff_BSM_test = np.zeros((len(x_test),5)), np.zeros((len(x_test),5))
    modelList = [('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' VXN RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' VXN RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' VXN RUT.h5 ' ),('vol10 RUT.h5','RUT SPX.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' VXN RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' VXN RUT.h5 ' ) ]
    errorList_csv = [('diff_test_RUT_1.csv',  'diff_BSM_test1.csv' ),('diff_test_RUT_2.csv', 'diff_BSM_test2.csv' ),('diff_test_RUT_3.csv',  'diff_BSM_test3.csv' ),('diff_test_RUT_4.csv', 'diff_BSM_test4.csv' ),('diff_test_RUT_5.csv', 'diff_BSM_test5.csv' )]
    scoreList_csv = [('mse_test1.csv','rmse_test1.csv','R2_test1.csv')]
    scoreListBSM_csv = [('mse_BSM_test1.csv','rmse_BSM_test1.csv','R2_BSM_test1.csv')]
    for i in range(5):


      

      X_train_sc = np.append(X_train, np.resize(vol_train[:,i], (len(vol_train),1)), axis = 1)
      X_test_sc = np.append(X_test, np.resize(vol_test[:,i], (len(vol_test),1)), axis = 1)
      print(X_test_sc)

      model = Sequential()
      model.add(layers.Dense(128, input_dim = 5))
      model.add(layers.Dense(128, activation = 'relu'))
      model.add(layers.Dense(128, activation = 'relu'))
      model.add(layers.Dense(1, activation = 'linear'))
      model.compile(loss='mse' ,optimizer='adam')
      model.summary()
      mod = model.fit(X_train_sc, y_train , epochs=500, batch_size=1024, validation_data=(X_test_sc, y_test), verbose = 1, callbacks=[earlyStop])
      y_test_hat = np.float64(scaler_y.inverse_transform(model.predict(X_test_sc)))
      training_loss = mod.history['loss']
      test_loss = mod.history['val_loss']
      out = y_test_hat.reshape(y_test_hat.shape[0])
      if i == 0:
        vol10['MLP']=pd.Series(out)
      if i == 1:
        vol30['MLP']=pd.Series(out)
      if i == 2:
        vol60['MLP']=pd.Series(out)
      if i == 3:
        ARIMA['MLP']=pd.Series(out)
      if i == 4:
        VXN['MLP']=pd.Series(out)                           
                
      # Create count of the number of epochs
      epoch_count = range(1, len(training_loss) + 1)

      # Visualize loss history
      plt.plot(epoch_count, training_loss, 'r--')
      plt.plot(epoch_count, test_loss, 'b-')
      plt.legend(['Training Loss', 'Test Loss'])
      plt.xlabel('Epoch')
      plt.ylabel('Loss')
      plt.show()
 
   


      mse_test[i] = mean_squared_error(y, y_test_hat)
      rmse_test[i] = math.sqrt(mse_test[i])
      R2_test[i] = r2_score(y, y_test_hat)
      mse_BSM_test[i] = mean_squared_error(y, BSM_test[:,i:i+1])
      rmse_BSM_test[i] = math.sqrt(mse_BSM_test[i])
      R2_BSM_test[i] = r2_score(y, BSM_test[:,i:i+1])      
      model.save(modelList[k][i])



    scoreList = [mse_test,rmse_test,R2_test]
    scoreListBSM = [mse_BSM_test,rmse_BSM_test,R2_BSM_test]
    
    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_MLP/'+ scoreList_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreList [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"VXN"})
      df.to_csv(filepath)
    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_MLP/'+ scoreListBSM_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreListBSM [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"VXN"})
      df.to_csv(filepath)
  return

### Defining GridSearchCV parameters for MLP



In [ ]:
# calling grid search
def mlpgridsearch():
  # define search space
  space = dict()
  space['hidden_layer'] = [(128,128,128),(128,64,128),(64,128,64),(64,64,64)]
  space['activation_function'] = ['tanh','relu']
  space['optimizer'] = ['SGD', 'Adam']
  search = GridSearchCV(mlp, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)

##CNN

In [ ]:
# defining the different models

def cnn(x_train,x_test,y_train,y_test,y,vol_train,vol_test,scaler_y,optimalvalues):
  from pathlib import Path  

  for k in range(1):
    mse_test,  mse_BSM_test = np.zeros(5), np.zeros(5)
    rmse_test  , rmse_BSM_test = np. zeros (5) , np. zeros (5) 
    R2_test  , R2_BSM_test = np. zeros (5) , np. zeros (5) 
    diff_test ,  diff_BSM_test = np.zeros((len(x_test),5)), np.zeros((len(x_test),5))
    modelList = [('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' VXN RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' VXN RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' VXN RUT.h5 ' ),('vol10 RUT.h5','RUT SPX.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' VXN RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' VXN RUT.h5 ' ) ]
    errorList_csv = [('diff_test_RUT_1.csv',  'diff_BSM_test1.csv' ),('diff_test_RUT_2.csv', 'diff_BSM_test2.csv' ),('diff_test_RUT_3.csv',  'diff_BSM_test3.csv' ),('diff_test_RUT_4.csv', 'diff_BSM_test4.csv' ),('diff_test_RUT_5.csv', 'diff_BSM_test5.csv' )]
    scoreList_csv = [('mse_test1.csv','rmse_test1.csv','R2_test1.csv')]
    scoreListBSM_csv = [('mse_BSM_test1.csv','rmse_BSM_test1.csv','R2_BSM_test1.csv')]
    for i in range(5):

      print('Fold', k+1, ',− Model', i+1)
      X_train_sc = np.append(X_train, np.resize(vol_train[:,i], (len(vol_train),1)), axis = 1)
      X_test_sc = np.append(X_test, np.resize(vol_test[:,i], (len(vol_test),1)), axis = 1)
      X_train_size = X_train_sc.shape[0] # number of samples in train set
      X_train_time_steps  = X_train_sc.shape[1] # number of features in train set
      input_dimension = 1               # each feature is represented by 1 number
      X_train_data_reshaped = X_train_sc.reshape(X_train_size,X_train_time_steps,input_dimension)

      X_test_size = X_test_sc.shape[0] # number of samples in train set
      X_test_time_steps  = X_test_sc.shape[1] # number of features in train set
      input_dimension = 1               # each feature is represented by 1 number
      X_test_data_reshaped = X_test_sc.reshape(X_test_size,X_test_time_steps,input_dimension)
      a=X_train_data_reshaped.shape[0]
      n_timesteps = X_train_data_reshaped.shape[1] #13
      n_features  = X_train_data_reshaped.shape[2] #1 
      print(n_features)


      #model = Sequential()
      model = keras.Sequential(name="model_conv1D")
      #model.add(tf.keras.layers.inputLayer({inputShape: [n_timesteps, n_features]}))
      model.add(keras.layers.Conv1D(64, 1,activation='relu',input_shape=(n_timesteps,n_features)))
      #model.add(layers.Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(n_timesteps, n_features)))
      model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
      #model.add(layers.Conv1D(filters=4, activation='relu'))
      #model.add(layers.MaxPooling1D(pool_size=2))
      model.add(layers.Flatten())
      
      model.add(layers.Dense(64, activation='elu'))
      model.add(layers.Dense(1))
      optimizer = tf.keras.optimizers.Adam(0.001)
      model.compile(loss='mse',optimizer=optimizer)
      plot_model(model, to_file='CNNnew.png', show_shapes=True, show_layer_names=True)
      model.summary()
      mod = model.fit(X_train_data_reshaped, y_train , epochs=50, batch_size=2048, validation_data=(X_test_data_reshaped, y_test), verbose = 1,callbacks=[earlyStop])
      y_test_hat = np.float64(scaler_y.inverse_transform(model.predict(X_test_data_reshaped)))
      training_loss = mod.history['loss']
      test_loss = mod.history['val_loss']
      out = y_test_hat.reshape(y_test_hat.shape[0])
      
      if i == 0:
        vol10['CNN']=pd.Series(out)
      if i == 1:
        vol30['CNN']=pd.Series(out)
      if i == 2:
        vol60['CNN']=pd.Series(out)
      if i == 3:
        ARIMA['CNN']=pd.Series(out)
      if i == 4:
        VXN['CNN']=pd.Series(out)                         
                
      # Create count of the number of epochs
      epoch_count = range(1, len(training_loss) + 1)

      # Visualize loss history
      plt.plot(epoch_count, training_loss, 'r--')
      plt.plot(epoch_count, test_loss, 'b-')
      plt.legend(['Training Loss', 'Test Loss'])
      plt.xlabel('Epoch')
      plt.ylabel('Loss')
      plt.show()

      mse_test[i] = mean_squared_error(y, y_test_hat)
      rmse_test[i] = math.sqrt(mse_test[i])
      R2_test[i] = r2_score(y, y_test_hat)
      mse_BSM_test[i] = mean_squared_error(y, BSM_test[:,i:i+1])
      rmse_BSM_test[i] = math.sqrt(mse_BSM_test[i])
      R2_BSM_test[i] = r2_score(y, BSM_test[:,i:i+1]) 
      model.save(modelList[k][i])
      print('Model', i+1, 'Completed') 

    print( 'Saving Residuals')
  
    scoreList = [mse_test,rmse_test,R2_test]
    scoreListBSM = [mse_BSM_test,rmse_BSM_test,R2_BSM_test]

    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_CNN/'+ scoreList_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreList [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"VXN"})
      df.to_csv(filepath)
    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_CNN/'+ scoreListBSM_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreListBSM [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"VXN"})
      df.to_csv(filepath)
  return


### Defining GridSearchCV parameters for CNN



In [ ]:
# calling grid search
def cnngridsearch():
  space = dict()
  space['batch_size'] = [512 , 1024 , 2048]
  space['activation_function'] = ['LeakyReLU','tanh','relu']
  space['optimizer'] = ['SGD', 'Adam']
  search = GridSearchCV(cnn, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
  return search

##LSTM

In [ ]:
# defining the different models

def lstm(x_train,x_test,y_train,y_test,y,vol_train,vol_test,scaler_y,optimalvalues):

  from pathlib import Path  

  for k in range(1):
    mse_test,  mse_BSM_test = np.zeros(5), np.zeros(5)
    rmse_test  , rmse_BSM_test = np. zeros (5) , np. zeros (5) 
    R2_test  , R2_BSM_test = np. zeros (5) , np. zeros (5) 
    diff_test ,  diff_BSM_test = np.zeros((len(x_test),5)), np.zeros((len(x_test),5))
    modelList = [('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','RUT SPX.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ) ]
    errorList_csv = [('diff_test_RUT_1.csv',  'diff_BSM_test1.csv' ),('diff_test_RUT_2.csv', 'diff_BSM_test2.csv' ),('diff_test_RUT_3.csv',  'diff_BSM_test3.csv' ),('diff_test_RUT_4.csv', 'diff_BSM_test4.csv' ),('diff_test_RUT_5.csv', 'diff_BSM_test5.csv' )]
    scoreList_csv = [('mse_test1.csv','rmse_test1.csv','R2_test1.csv')]
    scoreListBSM_csv = [('mse_BSM_test1.csv','rmse_BSM_test1.csv','R2_BSM_test1.csv')]
    for i in range(5):

      print('Fold', k+1, ',− Model', i+1)
      X_train_sc = np.append(X_train, np.resize(vol_train[:,i], (len(vol_train),1)), axis = 1)
      X_test_sc = np.append(X_test, np.resize(vol_test[:,i], (len(vol_test),1)), axis = 1)
      
      model = Sequential()
      model.add(keras.layers.LSTM(100,return_sequences = True,input_shape=(X_train_sc.shape[1], 1)))
      #model = Sequential()
      #model.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
    # model.add(layers.LSTM(100,  activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
      model.add(layers.Bidirectional(layers.LSTM(64))) # making LSTM bidirectional
      #model.add(layers.LSTM(units = 50,kernel_regularizer='l2',return_sequences = True,activation=keras.layers.LeakyReLU(alpha=0.01)))
      model.add(layers.Dropout(0.1))
      model.add(layers.Dense(1, activation='linear'))
      model.compile(optimizer='adam', loss='mse')
      model.summary()
      plot_model(model, to_file='LSTMnew.png', show_shapes=True, show_layer_names=True)
      mod = model.fit(X_train_sc, y_train , epochs=50, batch_size=2048, validation_data=(X_test_sc, y_test), verbose = 1,callbacks=[earlyStop])
      y_test_hat = np.float64(scaler_y.inverse_transform(model.predict(X_test_sc)))
      training_loss = mod.history['loss']
      test_loss = mod.history['val_loss']
      out = y_test_hat.reshape(y_test_hat.shape[0])
      if i == 0:
        vol10['LSTM']=pd.Series(out)
      if i == 1:
        vol30['LSTM']=pd.Series(out)
      if i == 2:
        vol60['LSTM']=pd.Series(out)
      if i == 3:
        ARIMA['LSTM']=pd.Series(out)
      if i == 4:
        RVX['LSTM']=pd.Series(out)                         
                
      # Create count of the number of epochs
      epoch_count = range(1, len(training_loss) + 1)

      # Visualize loss history
      plt.plot(epoch_count, training_loss, 'r--')
      plt.plot(epoch_count, test_loss, 'b-')
      plt.legend(['Training Loss', 'Test Loss'])
      plt.xlabel('Epoch')
      plt.ylabel('Loss')
      plt.show()
 
   

      mse_test[i] = mean_squared_error(y, y_test_hat)
      rmse_test[i] = math.sqrt(mse_test[i])
      R2_test[i] = r2_score(y, y_test_hat)
      mse_BSM_test[i] = mean_squared_error(y, BSM_test[:,i:i+1])
      rmse_BSM_test[i] = math.sqrt(mse_BSM_test[i])
      R2_BSM_test[i] = r2_score(y, BSM_test[:,i:i+1]) 
      model.save(modelList[k][i])
      print('Model', i+1, 'Completed') 

    print( 'Saving Residuals')
    
    scoreList = [mse_test,rmse_test,R2_test]
    scoreListBSM = [mse_BSM_test,rmse_BSM_test,R2_BSM_test]
    

    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_LSTM/'+ scoreList_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreList [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"RVX"})
      df.to_csv(filepath)
    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_LSTM/'+ scoreListBSM_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreListBSM [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"RVX"})
      df.to_csv(filepath)
  return


### Defining GridSearchCV parameters for LSTM



In [ ]:
# calling grid search
def lstmgridsearch():
  space = dict()
  space['batch_size'] = [512 , 1024 , 2048]
  space['activation_function'] = ['LeakyReLU','tanh','relu']
  space['optimizer'] = ['SGD', 'Adam']
  search = GridSearchCV(lstm, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
  return search

##CNN-LSTM

In [ ]:
# defining the different models

def cnnlstm(x_train,x_test,y_train,y_test,y,vol_train,vol_test,scaler_y,optimalvalues):
  from pathlib import Path  

  for k in range(1):
    mse_test,  mse_BSM_test = np.zeros(5), np.zeros(5)
    rmse_test  , rmse_BSM_test = np. zeros (5) , np. zeros (5) 
    R2_test  , R2_BSM_test = np. zeros (5) , np. zeros (5) 
    diff_test ,  diff_BSM_test = np.zeros((len(x_test),5)), np.zeros((len(x_test),5))
    modelList = [('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','RUT SPX.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ) ]
    errorList_csv = [('diff_test_RUT_1.csv',  'diff_BSM_test1.csv' ),('diff_test_RUT_2.csv', 'diff_BSM_test2.csv' ),('diff_test_RUT_3.csv',  'diff_BSM_test3.csv' ),('diff_test_RUT_4.csv', 'diff_BSM_test4.csv' ),('diff_test_RUT_5.csv', 'diff_BSM_test5.csv' )]
    scoreList_csv = [('mse_test1.csv','rmse_test1.csv','R2_test1.csv')]
    scoreListBSM_csv = [('mse_BSM_test1.csv','rmse_BSM_test1.csv','R2_BSM_test1.csv')]
    for i in range(5):
      subsequences = 1
      print('Fold', k+1, ',− Model', i+1)
      X_train_sc = np.append(X_train, np.resize(vol_train[:,i], (len(vol_train),1)), axis = 1)
      X_test_sc = np.append(X_test, np.resize(vol_test[:,i], (len(vol_test),1)), axis = 1)

      subsequences = 1
      X_train_size = X_train_sc.shape[0] # number of samples in train set
      X_train_time_steps  = X_train_sc.shape[1]
      train_timesteps = X_train_sc.shape[1]//subsequences  # number of features in train set
      #print(X_train_time_steps)
      input_dimension = 1  
 
      X_train_data_reshaped = X_train_sc.reshape(X_train_size,X_train_time_steps,input_dimension)
      #print(X_train_data_reshaped.shape)
      X_train_data_reshaped = X_train_data_reshaped.reshape((X_train_data_reshaped.shape[0], subsequences, train_timesteps, 1))
      
      subsequences = 1
      X_test_size = X_test_sc.shape[0] # number of samples in train set
      X_test_time_steps  = X_test_sc.shape[1]
      test_timesteps = X_test_sc.shape[1]//subsequences  # number of features in train set
      #print(X_train_time_steps)
      input_dimension = 1  
      #print(X_test_sc.shape)             # each feature is represented by 1 number
      X_test_data_reshaped = X_test_sc.reshape(X_test_size,X_test_time_steps,input_dimension)
      #print(X_test_data_reshaped.shape)
      X_test_data_reshaped = X_test_data_reshaped.reshape((X_test_data_reshaped.shape[0], subsequences, test_timesteps, 1))
      model= Sequential()
      model.add(keras.layers.TimeDistributed(keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_data_reshaped.shape[2], X_train_data_reshaped.shape[3])))
      #model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling1D(pool_size=2)))
      model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))
      model.add(keras.layers.LSTM(50, activation='elu'))
      model.add(keras.layers.Dense(1))
      model.compile(optimizer='adam', loss='mse')
      plot_model(model, to_file='CNNLSTMnew.png', show_shapes=True, show_layer_names=True)
      model.summary()
      mod = model.fit(X_train_data_reshaped, y_train , epochs=50, batch_size=2048, validation_data=(X_test_data_reshaped, y_test), verbose = 1,callbacks=[earlyStop])
      y_test_hat = np.float64(scaler_y.inverse_transform(model.predict(X_test_data_reshaped)))
      training_loss = mod.history['loss']
      test_loss = mod.history['val_loss']
      out = y_test_hat.reshape(y_test_hat.shape[0])
      if i == 0:
        vol10['CNNLSTM']=pd.Series(out)
      if i == 1:
        vol30['CNNLSTM']=pd.Series(out)
      if i == 2:
        vol60['CNNLSTM']=pd.Series(out)
      if i == 3:
        ARIMA['CNNLSTM']=pd.Series(out)
      if i == 4:
        RVX['CNNLSTM']=pd.Series(out)                         
                
      # Create count of the number of epochs
      epoch_count = range(1, len(training_loss) + 1)

      # Visualize loss history
      plt.plot(epoch_count, training_loss, 'r--')
      plt.plot(epoch_count, test_loss, 'b-')
      plt.legend(['Training Loss', 'Test Loss'])
      plt.xlabel('Epoch')
      plt.ylabel('Loss')
      plt.show()

      mse_test[i] = mean_squared_error(y, y_test_hat)
      rmse_test[i] = math.sqrt(mse_test[i])
      R2_test[i] = r2_score(y, y_test_hat)
      mse_BSM_test[i] = mean_squared_error(y, BSM_test[:,i:i+1])
      rmse_BSM_test[i] = math.sqrt(mse_BSM_test[i])
      R2_BSM_test[i] = r2_score(y, BSM_test[:,i:i+1]) 
      model.save(modelList[k][i])
      print('Model', i+1, 'Completed') 

    print( 'Saving Residuals')
    
    scoreList = [mse_test,rmse_test,R2_test]
    scoreListBSM = [mse_BSM_test,rmse_BSM_test,R2_BSM_test]


    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_CNNLSTM/'+ scoreList_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreList [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"RVX"})
      df.to_csv(filepath)
    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_CNNLSTM/'+ scoreListBSM_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreListBSM [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"RVX"})
      df.to_csv(filepath)
  return


### Defining GridSearchCV parameters for CNN-LSTM



In [ ]:
# calling grid search
def cnnlstmgridsearch():
  space = dict()
  space['batch_size'] = [512 , 1024 , 2048]
  space['activation_function'] = ['LeakyReLU','tanh','relu']
  space['optimizer'] = ['SGD', 'Adam']
  search = GridSearchCV(cnnlstm, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
  return search

## DUAL HYBRID


In [ ]:
def bsm_call(S, K, T, r, sigma):
    a = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    b = a - sigma * np.sqrt(T)
    return S * N(a) - K * np.exp(-r*T)* N(b)

def merton_jump_paths(S, T, r, sigma,  lam, m, v, steps, Npaths):
    size=(steps,Npaths)
    dt = T/steps 
    poi_rv = np.multiply(np.random.poisson( lam*dt, size=size),
                         np.random.normal(m,v, size=size)).cumsum(axis=0)
    geo = np.cumsum(((r -  sigma**2/2 -lam*(m  + v**2*0.5))*dt +\
                              sigma*np.sqrt(dt) * \
                              np.random.normal(size=size)), axis=0)
    
    return np.exp(geo+poi_rv)*S


from scipy.stats import norm
from scipy.optimize import minimize_scalar   
N = norm.cdf


def mjdcall(S, K, T, r, sigma, m , v, lam):
    p = 0
    for k in range(40):
        rk = r - lam*(m-1) + (k*np.log(m) ) / T
        sigmak = np.sqrt( sigma**2 + (k* v** 2) / T)
        Kfact = np.math.factorial(k)
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (Kfact))  * bsm_call(S, K, T, rk, sigmak)
    
    return p 
    
def optimal_params(x, a,mkt_prices, strikes):
    T = a['T'].values[i]
    S = a['Close'].values[i]
    r = a['riskefreerate'].values[i]
    price = a['price'].values[i] 
    candidate_prices = mjdcall(S, strikes, T, r,
                                        sigma=x[0], m= x[1] ,
                                        v=x[2],lam= x[3])
    return np.linalg.norm([mkt_prices - candidate_prices], 2)

def optimize_mjd(a,price,strikes):
  
      
  x0 = [0.15, 1, 0.1, 1] 
  bounds = ((0.01, np.inf) , (0.01, 2), (1e-5, np.inf) , (0, 5)) #bounds as described above

  res = minimize(optimal_params, method='SLSQP',  x0=x0, args=(a,price, strikes),bounds = bounds, tol=10e-4)
  sigt = res.x[0]
  mt = res.x[1]
  vt = res.x[2]
  lamt = res.x[3]
  return sigt,mt,vt,lamt

def dualmjd(a):
  deviation = []
  for i in range(len(a)):

    T = a['T'].values[i]
    S = a['Close'].values[i]
    r = a['riskefreerate'].values[i]
    price = a['price'].values[i]
    
    strike = a['strike_price'].values[i]
    sigt,mt,vt,lamt = optimize_mjd(a,price, strike)
    temp = mjdcall(S,strike,T,r,sigt,mt,vt,lamt)

    
    deviation.append(temp-price) 
    print(i+1)
  return deviation
def dualhybrid(df1,optimalvalues):
  df2 = df1.drop_duplicates(subset=["strike_price"], keep='first')
  x =dualmjd(df2)
  x = pd.DataFrame (x, columns = ['err'])
  x['date'] = df2.index
  x = x.set_index('date')
  df2 = df1.merge(x,left_index = True, right_index = True,how='left')
  print(df2)
  df2.dropna(inplace=True)
  X_train, X_test, y_train, y_test = train_test_split(df2[['T','M', 'riskefreerate' ,'rate','err','vol10','vol30','vol60','ARIMA','RVX','BSM (vol10)','BSM (vol30)','BSM (vol60)','BSM (Arima)','BSM (RVX)']].values, df2[['price']].values , test_size = 0.10, random_state = 110493)
  scaler_x = RobustScaler()
  scaler_y = RobustScaler()
  X_train_sc = scaler_x.fit_transform(X_train)
  X_test_sc = scaler_x.fit_transform(X_test)
  y_train_sc = scaler_y.fit_transform(y_train.reshape(-1, 1))
  y_test_sc = scaler_y.fit_transform(y_test.reshape(-1, 1))
  print(X_train)
  print(X_train_sc)
  BSM_train = np.resize(X_train[:,10:15], (len(X_train),5))
  BSM_test = np.resize(X_test[:,10:15], (len(X_test),5)) 
  vol_train = np.resize(X_train_sc[:,5:10], (len(X_train_sc),5))
  X_train = X_train_sc[: ,0:5]
  vol_test = np.resize(X_test_sc[: ,5:10] , (len(X_test_sc) ,5))
  X_test = X_test_sc[:,0:5]
  from pathlib import Path  

  for k in range(1):
    mse_test,  mse_BSM_test = np.zeros(5), np.zeros(5)
    rmse_test  , rmse_BSM_test = np. zeros (5) , np. zeros (5) 
    R2_test  , R2_BSM_test = np. zeros (5) , np. zeros (5) 
    diff_test ,  diff_BSM_test = np.zeros((len(X_test),5)), np.zeros((len(X_test),5))
    modelList = [('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','RUT SPX.h5','vol60 RUT.h5', 'ARIMA RUT.h5', ' RVX RUT.h5 ' ),('vol10 RUT.h5','vol30 RUT.h5','vol60 RUT.h5','ARIMA RUT.h5', ' RVX RUT.h5 ' ) ]
    errorList_csv = [('diff_test_RUT_1.csv',  'diff_BSM_test1.csv' ),('diff_test_RUT_2.csv', 'diff_BSM_test2.csv' ),('diff_test_RUT_3.csv',  'diff_BSM_test3.csv' ),('diff_test_RUT_4.csv', 'diff_BSM_test4.csv' ),('diff_test_RUT_5.csv', 'diff_BSM_test5.csv' )]
    scoreList_csv = [('mse_test1.csv','rmse_test1.csv','R2_test1.csv')]
    scoreListBSM_csv = [('mse_BSM_test1.csv','rmse_BSM_test1.csv','R2_BSM_test1.csv')]
    for i in range(5):
      subsequences = 1
      print('Fold', k+1, ',− Model', i+1)
      X_train = np.append(X_train_sc, np.resize(vol_train[:,i], (len(vol_train),1)), axis = 1)
      X_test = np.append(X_test_sc, np.resize(vol_test[:,i], (len(vol_test),1)), axis = 1)

      subsequences = 1
      X_train_size = X_train.shape[0] # number of samples in train set
      X_train_time_steps  = X_train.shape[1]
      train_timesteps = X_train.shape[1]//subsequences  # number of features in train set
      #print(X_train_time_steps)
      input_dimension = 1  
 
      X_train_data_reshaped = X_train.reshape(X_train_size,X_train_time_steps,input_dimension)
      #print(X_train_data_reshaped.shape)
      X_train_data_reshaped = X_train_data_reshaped.reshape((X_train_data_reshaped.shape[0], subsequences, train_timesteps, 1))
      
      subsequences = 1
      X_test_size = X_test.shape[0] # number of samples in train set
      X_test_time_steps  = X_test.shape[1]
      test_timesteps = X_test.shape[1]//subsequences  # number of features in train set
      #print(X_train_time_steps)
      input_dimension = 1  
      #print(X_test_sc.shape)             # each feature is represented by 1 number
      X_test_data_reshaped = X_test.reshape(X_test_size,X_test_time_steps,input_dimension)
      #print(X_test_data_reshaped.shape)
      X_test_data_reshaped = X_test_data_reshaped.reshape((X_test_data_reshaped.shape[0], subsequences, test_timesteps, 1))
      model= Sequential()
      model.add(keras.layers.TimeDistributed(keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_data_reshaped.shape[2], X_train_data_reshaped.shape[3])))
      #model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling1D(pool_size=2)))
      model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))
      model.add(keras.layers.LSTM(50, activation='elu'))
      model.add(keras.layers.Dense(1))
      model.compile(optimizer='adam', loss='mse')
      model.summary()
      mod = model.fit(X_train_data_reshaped, y_train_sc , epochs=50, batch_size=2048, validation_data=(X_test_data_reshaped, y_test_sc), verbose = 1,callbacks=[earlyStop])
      y_test_hat = np.float64(scaler_y.inverse_transform(model.predict(X_test_data_reshaped)))
      training_loss = mod.history['loss']
      test_loss = mod.history['val_loss']
      out = y_test_hat.reshape(y_test_hat.shape[0])
      if i == 0:
        vol10['MJD_CNNLSTM']=pd.Series(out)
      if i == 1:
        vol30['MJD_CNNLSTM']=pd.Series(out)
      if i == 2:
        vol60['MJD_CNNLSTM']=pd.Series(out)
      if i == 3:
        ARIMA['MJD_CNNLSTM']=pd.Series(out)
      if i == 4:
        RVX['MJD_CNNLSTM']=pd.Series(out)                         
                
      # Create count of the number of epochs
      epoch_count = range(1, len(training_loss) + 1)

      # Visualize loss history
      plt.plot(epoch_count, training_loss, 'r--')
      plt.plot(epoch_count, test_loss, 'b-')
      plt.legend(['Training Loss', 'Test Loss'])
      plt.xlabel('Epoch')
      plt.ylabel('Loss')
      plt.show()

      mse_test[i] = mean_squared_error(y_test, y_test_hat)
      rmse_test[i] = math.sqrt(mse_test[i])
      R2_test[i] = r2_score(y_test, y_test_hat)
      mse_BSM_test[i] = mean_squared_error(y_test, BSM_test[:,i:i+1])
      rmse_BSM_test[i] = math.sqrt(mse_BSM_test[i])
      R2_BSM_test[i] = r2_score(y_test, BSM_test[:,i:i+1]) 
      model.save(modelList[k][i])
      print('Model', i+1, 'Completed') 

    print( 'Saving Residuals')
    
    scoreList = [mse_test,rmse_test,R2_test]
    scoreListBSM = [mse_BSM_test,rmse_BSM_test,R2_BSM_test]

    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_MJD_CNNLSTM/'+ scoreList_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreList [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"RVX"})
      df.to_csv(filepath)
    for j in range(3):
      filepath = Path('/content/drive/MyDrive/RUT_MJD_CNNLSTM/'+ scoreListBSM_csv[k][j])  
      filepath.parent.mkdir(parents=True, exist_ok=True) 
      df = pd.DataFrame( scoreListBSM [j])
      df = df.rename(columns={"0": "vol10", "1": "vol30", "2": "vol60", "3": "ARIMA", "4":"RVX"})
      df.to_csv(filepath)
  return
 

### Defining GridSearchCV parameters for Jump-Diffusion CNN-LSTM



In [ ]:
# calling grid search
def dualmjdgridsearch():
  space = dict()
  space['batch_size'] = [512 , 1024 , 2048]
  space['activation_function'] = ['LeakyReLU','tanh','relu']
  space['optimizer'] = ['SGD', 'Adam']
  search = GridSearchCV(dualmjd, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
  return search

## RUT


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DATA/RUT.csv')

In [ ]:
df = df.drop(columns=['secid','symbol','cp_flag','optionid','am_set_flag'])
df['strike_price'] = df['strike_price']/1000

In [ ]:
df['date'] = pd.to_datetime(df['date'], format = '%Y%m%d')
df['exdate'] = pd.to_datetime(df['exdate'], format = '%Y%m%d')
df['T'] = ((df['exdate'] - df['date']).dt.days)/365 

In [ ]:
pip install  yfinance

In [ ]:
pip install yfinance --upgrade --no-cache-dir

In [ ]:
import yfinance as yf
import pandas as pd
import datetime

In [ ]:
c = yf.download("^RUT", start='2012-01-01', end='2022-01-01')
c = c.drop(columns=['High','Open','Low','Adj Close','Volume'])

In [ ]:
c.dropna(inplace=True)
df.dropna(inplace = True)


In [ ]:
df = df.set_index('date')

In [ ]:
df1 = df.merge(c , left_index =True, right_index = True, how = 'left')

In [ ]:
df1['price'] = ((df['best_bid'] + df['best_offer']))/2 

In [ ]:
riskfree = pd.read_csv('/content/drive/MyDrive/DATA/10YM.csv',parse_dates = True, index_col = 'DATE')

In [ ]:
riskfree.dropna(inplace=True)

In [ ]:

riskfree['DGS10'] = riskfree['DGS10'].replace(".","", regex=False)

riskfree['DGS10'] = riskfree['DGS10'].replace("",np.nan, regex=True)

In [ ]:
riskfree.dropna(inplace=True)

In [ ]:
riskfree = riskfree.rename(columns={"DGS10": "riskefreerate"})

In [ ]:
df1 = df1.merge(riskfree , left_index =True, right_index = True, how = 'left')

In [ ]:
df1.dropna(inplace=True)

In [ ]:
df1['riskefreerate'] =df1['riskefreerate'].astype(float)

In [ ]:
volindex = pd.read_csv('/content/drive/MyDrive/DATA/RVX.csv', parse_dates = True, index_col = 'Date')

In [ ]:
df1 = df1.merge(volindex , left_index =True, right_index = True, how = 'left')

In [ ]:
df1 = df1.rename(columns={"RVX": "RVX"})

In [ ]:
df1['RVX'] = df1['RVX'].astype(float)
df1['RVX'] = df1['RVX']/100

In [ ]:
df1['riskefreerate'] =df1['riskefreerate']/100


In [ ]:
df1['Close'] = df1['Close'].replace(",","", regex=True)

In [ ]:
df1['Close'] = df1['Close'].astype(float)

In [ ]:
df1.dropna(inplace=True)

In [ ]:
c = yf.Ticker("^RVX")
c.info

In [ ]:
c = yf.download("^RUT", start='2002-01-01', end='2022-01-01')
c = c.drop(columns=['High','Open','Low','Adj Close','Volume'])

In [ ]:
c

In [ ]:
c['return'] = c['Close'].pct_change()
c.dropna(inplace=True)


In [ ]:
c = c.drop(columns=['Close'])

In [ ]:



# Historical Volatility (Rolling)
vol = pd.DataFrame(columns = ['vol10','vol30','vol60'])
vol['vol10'] = c['return'].rolling(window = 10).std()*np.sqrt(252)
vol['vol30'] = c['return'].rolling(window = 30).std()*np.sqrt(252)
vol['vol60'] = c['return'].rolling(window = 60).std()*np.sqrt(252)
vol.dropna(inplace=True)

In [ ]:
import datetime as dt

In [ ]:

start = c.index.searchsorted(dt.datetime(2012, 1, 1))

end = c.index.searchsorted(dt.datetime(2021, 12, 31))

y = c.iloc[start:end]


In [ ]:

start = c.index.searchsorted(dt.datetime(2002, 1, 1))

end = c.index.searchsorted(dt.datetime(2011, 12, 31))

x = c.iloc[start:end]


In [ ]:
# evaluate an ARIMA model using a walk-forward validation
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
# load dataset

# split into train and test sets
X = x.values
X = X*100
train = X[:]
Y = y.values
test = Y[:] 
history = [x for x in train]
predictions = list()
# walk-forward validation
i = 0
for t in range(len(test)):
    
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
  #preditctions
	obs = test[t]
	history.append(obs)
	print(t)


# evaluate forecasts
df = pd.DataFrame(predictions)
df = df.astype(float)

df = abs(df)
df = np.sqrt(df) 
df = df.set_index(y.index)
vol['ARIMA']=df
#rmse = sqrt(mean_squared_error(test, predictions))
#print('Test RMSE: %.3f' % rmse)
# plot forecasts against actual outcomes
#pyplot.plot(test)
#pyplot.plot(predictions, col)


In [ ]:
vol.dropna(inplace = True)

In [ ]:
df1 = df1.merge(vol, left_index=True, right_index=True, how = 'left')

In [ ]:
df1.dropna(inplace=True)

In [ ]:
df1 = df1.drop(columns=['issuer','ticker','index_flag','exercise_style','exdate'])

In [ ]:

# Continous Dividends (q)
q = pd.read_csv('/content/drive/MyDrive/DATA/RUTdiv.csv', parse_dates= True )
q['date'] = pd.to_datetime(q['date'], format = '%Y%m%d')
q = q.set_index('date')
q['rate']=q['rate']/100

q = pd.DataFrame(q['rate'])
df1 = df1.merge(q, left_index=True, right_index=True, how='left')



In [ ]:
df1['BSM (vol10)'] = (bsmcall(df1['Close'],df1['strike_price'],df1['T'],df1['riskefreerate'],df1['rate'],df1['vol10'])) / df1['strike_price']
df1['BSM (vol30)'] = (bsmcall(df1['Close'],df1['strike_price'],df1['T'],df1['riskefreerate'],df1['rate'],df1['vol30'])) / df1['strike_price']
df1['BSM (vol60)'] = (bsmcall(df1['Close'],df1['strike_price'],df1['T'],df1['riskefreerate'],df1['rate'],df1['vol60'])) / df1['strike_price']
df1['BSM (Arima)'] = (bsmcall(df1['Close'],df1['strike_price'],df1['T'],df1['riskefreerate'],df1['rate'],df1['ARIMA'])) / df1['strike_price']
df1['BSM (RVX)'] = (bsmcall(df1['Close'],df1['strike_price'],df1['T'],df1['riskefreerate'],df1['rate'],df1['RVX'])) / df1['strike_price']

In [ ]:
df1['M'] = df1['Close']/df1['strike_price']
df1 = df1[(df1['M'] <= 1.5)] 
df1 = df1[(df1['M'] >= 0.8)]

In [ ]:
df1['price'] = df1['price'] / df1['strike_price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df1[['T','M', 'riskefreerate' ,'rate','RVX','vol10','vol30','vol60','ARIMA','BSM (vol10)','BSM (vol30)','BSM (vol60)','BSM (Arima)','BSM (RVX)','strike_price']].values, df1[['price']].values , test_size = 0.10, random_state = 110493)

In [ ]:
scaler_x = RobustScaler()
scaler_y = RobustScaler()
X_train_sc = scaler_x.fit_transform(X_train)
X_test_sc = scaler_x.fit_transform(X_test)
y_train_sc = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_sc = scaler_y.fit_transform(y_test.reshape(-1, 1))
BSM_train = np.resize(X_train[:,9:14], (len(X_train),5))
X_train = X_train[: ,0:5]
strike_test = np.resize(X_test[:,14:15], (len(X_test),1)) 
BSM_test = np.resize(X_test[:,9:14], (len(X_test),5)) 
X_test =X_test[:,0:5]
vol_train = np.resize(X_train_sc[:,4:9], (len(X_train_sc),5))
X_train = X_train_sc[: ,0:4]
vol_test = np.resize(X_test_sc[: ,4:9] , (len(X_test_sc) ,5))
X_test = X_test_sc[:,0:4]

In [ ]:
vol10 = pd.DataFrame()
vol30 = pd.DataFrame()
vol60 = pd.DataFrame()
ARIMA = pd.DataFrame()
RVX = pd.DataFrame()

In [ ]:
import tensorflow as tf

## Caling gridsearch and respective model.


In [ ]:
optimalvalues = mlpgridsearch()

In [ ]:
mlp(X_train,X_test,y_train_sc,y_test_sc,y_test,vol_train,vol_test,scaler_y,BSM_train,BSM_test,optimalvalues)

In [ ]:
optimalvalues = dualmjdgridsearch()

In [ ]:
dualhybrid(df1,optimalvalues)

In [ ]:
optimalvalues = cnngridsearch()

In [ ]:
cnn(X_train_sc,X_test_sc,y_train_sc,y_test_sc,y_test,vol_train,vol_test,scaler_y,optimalvalues)

In [ ]:
optimalvalues = lstmgridsearch()

In [ ]:
lstm(X_train_sc,X_test_sc,y_train_sc,y_test_sc,y_test,vol_train,vol_test,scaler_y,optimalvalues)

In [ ]:
optimalvalues = cnnlstmgridsearch()

In [ ]:

cnnlstm(X_train_sc,X_test_sc,y_train_sc,y_test_sc,y_test,vol_train,vol_test,scaler_y,optimalvalues)